In [2]:
import pandas as pd

In [4]:
dtypes_list= {'qLegalStatus': 'str', 'qSpecies': 'str', 'qAddress': 'str', 'qSiteInfo': 'str', 'qCaretaker': 'str', 'PlantDate': 'str', 'PlotSize': 'str'}
trees_filtered = pd.read_csv('Street_Tree_List-2022-01-30_FILTERED.csv', dtype = dtypes_list)
trees_raw = pd.read_csv('Street_Tree_List-2022-01-30_RAW.csv', dtype = dtypes_list)

In [5]:
trees_filtered

,TreeID,qLegalStatus,qSpecies,qAddress,SiteOrder,qSiteInfo,qCaretaker,PlantDate,DBH,PlotSize,Latitude,Longitude
0,200311,DPW Maintained,Liquidambar styraciflua 'Rotundiloba' :: Round...,300 Spear St,9.0,Sidewalk: Curb side : Yard,Private,NaN,5,NaN,37.789193,-122.390006
1,10425,DPW Maintained,Schinus terebinthifolius :: Brazilian Pepper,1419 Haight St,2.0,Sidewalk: Curb side : Cutout,DPW,NaN,90,3x3,37.770094,-122.445693
2,43163,DPW Maintained,Acacia melanoxylon :: Blackwood Acacia,222 Madison St,1.0,Sidewalk: Curb side : Cutout,Private,7/11/00 0:00,8,NaN,37.726036,-122.422690
3,188921,DPW Maintained,Callistemon citrinus :: Lemon Bottlebrush,2001 25th St,16.0,Sidewalk: Property side : Yard,Private,NaN,6,Width 8ft,37.751955,-122.399482
4,643,DPW Maintained,Ficus microcarpa :: Chinese Banyan,3224 24th St,1.0,Sidewalk: Curb side : Cutout,Private,NaN,18,3x3,37.752398,-122.416772
5,195969,DPW Maintained,Populus nigra 'Italica' :: Lombardy Poplar,200 Tulare St,15.0,Back Yard : Yard,Private,NaN,8,NaN,37.747961,-122.389434
6,133773,DPW Maintained,Platanus x hispanica :: Sycamore: London Plane,2460 Scott St,1.0,Sidewalk: Curb side : Cutout,Private,NaN,16,Width 4ft,37.792467,-122.439533
7,137657,DPW Maintained,Crataegus laevigata :: English Hawthorn,2167 TURK BLVD,1.0,Sidewalk: Curb side : Cutout,Private,NaN,6,Width 0ft,37.778804,-122.444757
8,209974,DPW Maintained,Metrosideros excelsa :: New Zealand Xmas Tree,634 Miramar Ave,1.0,Sidewalk: Curb side : Cutout,Private,NaN,6,Width 3ft,37.728313,-122.458106
9,9999,DPW Maintained,Ficus microcarpa nitida 'Green Gem' :: Indian ...,126 Guerrero St,1.0,Sidewalk: Curb side : Cutout,DPW,NaN,18,5x12,37.769212,-122.424656


In [6]:
trees_filtered.dtypes

TreeID            int64
qLegalStatus     object
qSpecies         object
qAddress         object
SiteOrder       float64
qSiteInfo        object
qCaretaker       object
PlantDate        object
DBH               int64
PlotSize         object
Latitude        float64
Longitude       float64
dtype: object

In [18]:
print('total trees: ' + str(len(trees_filtered)))
a = trees_filtered['qSpecies'].value_counts()
print('number of species with only 1 tree: ' + str(len(a[a == 1])))
print('total number of species: ' + str(trees_filtered['qSpecies'].nunique()))

total trees: 37118
number of species with only 1 tree: 77
total number of species: 419


In [21]:
a

Platanus x hispanica :: Sycamore: London Plane                               2633
Metrosideros excelsa :: New Zealand Xmas Tree                                1937
Lophostemon confertus :: Brisbane Box                                        1727
Pittosporum undulatum :: Victorian Box                                       1669
Tristaniopsis laurina :: Swamp Myrtle                                        1577
Ficus microcarpa nitida 'Green Gem' :: Indian Laurel Fig Tree 'Green Gem'    1425
Magnolia grandiflora :: Southern Magnolia                                    1377
Prunus cerasifera :: Cherry Plum                                             1306
Arbutus 'Marina' :: Hybrid Strawberry Tree                                   1063
Prunus serrulata 'Kwanzan' :: Kwanzan Flowering Cherry                        841
Acacia melanoxylon :: Blackwood Acacia                                        820
Olea europaea :: Olive Tree                                                   789
Corymbia ficifol

In [20]:
trees_filtered[trees_filtered.qSpecies == 'Ficus microcarpa :: Chinese Banyan']

,TreeID,qLegalStatus,qSpecies,qAddress,SiteOrder,qSiteInfo,qCaretaker,PlantDate,DBH,PlotSize,Latitude,Longitude
4,643,DPW Maintained,Ficus microcarpa :: Chinese Banyan,3224 24th St,1.0,Sidewalk: Curb side : Cutout,Private,NaN,18,3x3,37.752398,-122.416772
79,226474,DPW Maintained,Ficus microcarpa :: Chinese Banyan,3160 19th St,1.0,Sidewalk: Curb side : Cutout,Private,NaN,20,Width 3ft,37.760609,-122.413957
281,218820,DPW Maintained,Ficus microcarpa :: Chinese Banyan,205 Beverly St,2.0,Sidewalk: Property side : Yard,Private,NaN,4,Width 5ft,37.719801,-122.471735
340,224612,DPW Maintained,Ficus microcarpa :: Chinese Banyan,525 Burnett Ave,2.0,Sidewalk: Curb side : Cutout,Private,NaN,7,Width 2ft,37.752857,-122.444682
350,26548,DPW Maintained,Ficus microcarpa :: Chinese Banyan,3085 24th St,4.0,Sidewalk: Curb side : Cutout,DPW,NaN,18,3X3,37.752464,-122.413583
987,184317,DPW Maintained,Ficus microcarpa :: Chinese Banyan,1557 39th Ave,6.0,Sidewalk: Curb side : Cutout,Private,NaN,8,Width 0ft,37.757297,-122.498259
1132,590,DPW Maintained,Ficus microcarpa :: Chinese Banyan,3049 24th St,1.0,Sidewalk: Curb side : Cutout,DPW,NaN,12,3X3,37.752501,-122.413169
1516,224506,DPW Maintained,Ficus microcarpa :: Chinese Banyan,2404 Folsom St,1.0,Sidewalk: Curb side : Cutout,Private,NaN,16,Width 4ft,37.758759,-122.414784
1543,224613,DPW Maintained,Ficus microcarpa :: Chinese Banyan,525 Burnett Ave,3.0,Sidewalk: Curb side : Cutout,Private,NaN,7,Width 2ft,37.752884,-122.444666
1594,105684,Property Tree,Ficus microcarpa :: Chinese Banyan,635 08TH ST,1.0,Front Yard : Pot,Private,NaN,10,NaN,37.771824,-122.403882
